<a href="https://colab.research.google.com/github/lukasjacon/TD-IDF-e-COSSENO/blob/main/TD_IDF_e_COSSENO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lukas Jacon Barboza

1. Sua tarefa será gerar a matriz termo-documento usando TF-IDF por meio da aplicação das fórmulas  TF-IDF  na  matriz termo-documento  criada com a utilização do algoritmo  Bag of Words. Sobre o Corpus que recuperamos anteriormente. O entregável desta tarefa é uma matriz termo-documento onde a primeira linha são os termos e as linhas subsequentes são os vetores calculados com o TF-IDF. <br>
2. Sua tarefa será gerar uma matriz de distância, computando o cosseno do ângulo entre todos os vetores que encontramos usando o tf-idf. Para isso use a seguinte fórmula para o cálculo do  cosseno use a fórmula apresentada em Word2Vector frankalcantara.com) (https://frankalcantara.com/Aulas/Nlp/out/Aula4.html#/0/4/2) e apresentada na figura a seguir:  <br>

In [22]:
from bs4 import BeautifulSoup
import requests
import spacy
import string
import numpy as np

nlp = spacy.load("en_core_web_sm")
nlp.add_pipe('sentencizer')

sites = [
  'https://www.datarobot.com/blog/what-is-natural-language-processing-introduction-to-nlp/',
  'https://www.qualtrics.com/experience-management/customer/natural-language-processing/', 
  'https://www.oanayucel.ro/en/nlp-what-is-it-and-how-can-it-help-you/', 
  'https://viso.ai/deep-learning/natural-language-processing/',
  'https://www.oracle.com/hk/artificial-intelligence/what-is-natural-language-processing/'
]
corpuses = []

for site in sites:
  pagina = requests.get(site)
  if pagina.status_code != 200:
    print(f"Ocorreu um erro ao buscar a página {site}.");
    continue
  soup = BeautifulSoup(pagina.content, 'html.parser') 
  corpus = []
  for tagParagrafo in soup.find_all(["p", "h1", "h2", "h3", "h4", "h5", "h6", "li" ]): 
    paragrafo = tagParagrafo.get_text()
    strippedParagrafo = paragrafo.strip()
    if len(strippedParagrafo) > 0:
      doc = nlp(strippedParagrafo)
      for sentença in doc.sents:
        corpus.append(sentença.text.translate(str.maketrans('', '', string.punctuation)))
  corpuses.append(corpus)
  #print(f"Número de palavras em {site}: {len(' '.join(corpus).split(' '))}")
  #print(corpus)

In [23]:
lexemas = []
numeroSentencas = 0
for corpus in corpuses:
  for sentencas in corpus:
    numeroSentencas += 1
    for lexema in sentencas.split(' '):
      if lexema not in lexemas:
        lexemas.append(lexema)
bagOfWords = np.zeros((numeroSentencas,len(lexemas)))
sentencaAtual = 0
for corpus in corpuses:
  for sentencas in corpus:
    for lexema in sentencas.split(' '):
      bagOfWords[sentencaAtual][lexemas.index(lexema)] += 1
    sentencaAtual += 1

#print(f"Sentenças: {len(bagOfWords)}")
#print(f"Lexemas distintos: {len(bagOfWords[0])}")
#print("Bag of Words: ")
#print(bagOfWords)

Parte 1 - TF - IDF

In [24]:
# TF
tfs = np.zeros((len(bagOfWords),len(bagOfWords[0])))
sentencaAtual = 0
for corpus in corpuses:
  for sentencas in corpus:
    numeroDeLexemas = len(sentencas.split(' '))
    for lexema in sentencas.split(' '):
      tfs[sentencaAtual][lexemas.index(lexema)] = bagOfWords[sentencaAtual][lexemas.index(lexema)] / numeroDeLexemas
    sentencaAtual += 1
print("TF: ")
print(tfs)

# IDF
idfs = []
for lexema in range(len(lexemas)):
  lexemeRate = 0
  for sentenceRates in bagOfWords:
    if sentenceRates[lexema] > 0: lexemeRate += 1
  idfs.append(np.log10(len(bagOfWords)/lexemeRate))
print("IDF: ")
print(len(bagOfWords))
print(idfs)

tfidf = np.zeros((len(bagOfWords),len(bagOfWords[0])))
for i in range(len(tfidf)):
  for j in range(len(tfidf[0])):
    tfidf[i][j] = tfs[i][j] * idfs[j]
print('TF-IDF: ')
print(tfidf)

TF: 
[[0.25       0.25       0.25       ... 0.         0.         0.        ]
 [0.         0.         0.06451613 ... 0.         0.         0.        ]
 [0.         0.09090909 0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         1.         0.        ]
 [0.         0.         0.         ... 0.         0.         1.        ]]
IDF: 
2301
[3.3619166186686433, 0.7631261119055284, 1.4481027662849268, 3.3619166186686433, 3.3619166186686433, 2.759856627340681, 3.3619166186686433, 2.0001887826510503, 1.0501627576128891, 3.3619166186686433, 3.3619166186686433, 3.3619166186686433, 3.3619166186686433, 3.3619166186686433, 3.3619166186686433, 3.3619166186686433, 3.3619166186686433, 3.3619166186686433, 3.3619166186686433, 3.3619166186686433, 2.3205239335104184, 3.3619166186686433, 3.3619166186686433, 3.3619166186686433, 2.6629466143326246, 3.3619166186686433, 1.9639766099966058, 

Parte 2 - Matriz de Distâncias

In [25]:
currentVetor = 0
matrizDeDistancia = np.zeros((len(tfidf),len(tfidf)))

for vetor in tfidf:
  i = currentVetor
  while i < len(tfidf):
    distance = np.dot(vetor,tfidf[i])/(np.linalg.norm(vetor)*np.linalg.norm(tfidf[i]))
    matrizDeDistancia[currentVetor][i] = distance
    matrizDeDistancia[i][currentVetor] = distance
    i += 1
  currentVetor += 1

print(matrizDeDistancia)

[[1.         0.04991406 0.01373262 ... 0.         0.         0.        ]
 [0.04991406 1.         0.20645153 ... 0.         0.         0.        ]
 [0.01373262 0.20645153 1.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 1.         0.         0.        ]
 [0.         0.         0.         ... 0.         1.         0.        ]
 [0.         0.         0.         ... 0.         0.         1.        ]]
